IMPORTING DATA FROM QUANDL USING QUANDL API

In [8]:
import pandas as pd
import math
import quandl
data = quandl.get("WIKI/GOOGL")
data.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [9]:
data= data[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
data.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0


In [10]:
data['HL_PCT'] = (data['Adj. High']-data['Adj. Close'])/data['Adj. Close']*100.00
data['PCT_change'] = (data['Adj. Close']-data['Adj. Open'])/data['Adj. Open']*100.00
data.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,HL_PCT,PCT_change
Date,,,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0,3.712563,0.324968
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0,0.710922,7.227007
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0,3.729433,-1.227880
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0,6.417469,-5.726357
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0,1.886792,1.183658


In [11]:
data = data[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']]
forcast_col = 'Adj. Close'
data.fillna(-99999, inplace = True)
forcast_out = int(math.ceil(0.1*len(data)))
print(len(data))
print(forcast_out)
data['label'] = data[forcast_col].shift(-forcast_out)
data.dropna(inplace=True)
data.head()
data.tail()

3424
343


,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2016-11-04,781.10,0.944821,1.270582,1970603.0,1094.00
2016-11-07,802.03,0.370310,0.890622,1992570.0,1053.15
2016-11-08,811.98,0.500012,1.240602,1769069.0,1026.55
2016-11-09,805.59,0.759692,0.468927,3098510.0,1054.09
2016-11-10,780.29,3.815248,-3.667901,5909609.0,1006.94


# Model building and checking accuracy

In [16]:
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
X = np.array(data.drop(['label'],1))
X = preprocessing.scale(X)
X_lately = X[-forcast_out:]
X = X[:-forcast_out]
y = np.array(data['label'][:-forcast_out])
#y = np.array(data['label'])
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)
clf = LinearRegression()
clf.fit(X_train, y_train)
Accuracy  = clf.score(X_test, y_test)
Forcast_set = clf.predict(X_lately)
print(Forcast_set, Accuracy, forcast_out)

[664.46683816 661.85919622 659.22919857 675.39284918 698.94426264
 716.9097489  707.83016511 729.52077436 833.01251471 827.82205183
 848.74902676 854.4474243  823.06933936 799.78555261 803.41842901
 790.30679569 788.24332551 801.95448662 789.25992313 799.78352336
 796.45761033 818.40474951 800.28890849 794.98065041 800.46327481
 829.38407843 823.28514628 825.61627644 827.90288994 829.87778521
 828.22346833 836.04448682 827.38022966 794.68334739 800.9038173
 768.29398719 799.95969203 798.81037422 792.59965042 795.19880958
 774.71036642 772.70644242 776.67504306 758.8933745  770.23549346
 784.82935058 788.70290347 783.23895579 779.66915444 804.6008305
 795.77746467 819.69858298 795.35296463 796.62778254 781.66316105
 790.99343107 786.94369948 780.65524081 766.97690846 763.23896339
 766.01465076 770.3250596  799.69887001 808.61026318 812.22323859
 804.6768012  798.67952468 805.7047276  812.67698945 827.48339683
 822.73456663 829.15919658 825.32250175 835.22192724 823.87259342
 812.6885401